# Controlling a microcontroller

The purpose of this demonstration is to show one way to control a microcontroller via a python program running on a host computer.  The required setup:

1. A computer that has python
2. A microcontroller running circuitpython
3. The two are connected with a USB cable

## Programs

There are three programs required to make this function. 

1. A python script running on the HOST computer
2. A code.py script running on the MCU 
3. A boot.py script running on the MCU

The third is the easiest. It is only a couple of lines and ensures that the MCU is set up to send and receive serial commands. If it is not present, create a `boot.py` file on the MCU with the following code

In [ ]:
import usb_cdc

usb_cdc.enable(console=True, data=True)

The `code.py` script below is generic, so make sure you save anything in code.py that you would like to keep before proceeding.  This script will accept two different commands: "ON" and "OFF". Depending on the command received, it will turn the builtin LED on or off. It also checks to see if the LED is on or off and will warn the user if she is trying to turn the LED off when it is already off, or on when it is already on. It will also capture any unknown commands and respond appropriately.

In [ ]:
import board
import digitalio
import usb_cdc

led = digitalio.DigitalInOut(board.D13)
led.direction = digitalio.Direction.OUTPUT



def read_serial():
    if usb_cdc.data.in_waiting > 0:
        return usb_cdc.data.readline().decode().strip()
    return None

while True:
    command = read_serial()
    if command:
        print(f"Received command: {command}")  # Debug print
        if command == 'ON':
            if led.value:
                usb_cdc.data.write(b"Warning: The LED is already on.\n")
            else:
                led.value = True
                usb_cdc.data.write(b"LED is now ON.\n")
        elif command == 'OFF':
            if not led.value:
                usb_cdc.data.write(b"Warning: The LED is already off.\n")
            else:
                led.value = False
                usb_cdc.data.write(b"LED is now OFF.\n")
        else:
            usb_cdc.data.write(b"Command not understood.\n")

The program on the HOST is a set of three functions. One sends a serial command, one receives a response from the MCU, and one runs the main script. The structure demonstrates how one can create a "program" in python, which could be run by typing something like `python -m myprogram` on the command line.  The program also does some advanced things worth exploring:

1. It uses the argparse library to accept the COM port on the command line.  Therefore, it is not necessary to hard code the port into the program.
2. It handles the keyboard interrupt so that the program exits cleanly when the user presses CTRL-C.

In [ ]:
import serial
import time
import argparse

def send_command(command):
	ser.write((command + '\n').encode())
	time.sleep(0.5)

def get_response():
	response = ser.readline().decode().strip()
	if response:
		print(f"MCU Response: {response}")
	else:
		print("No response received from MCU.")

def main():
	parser = argparse.ArgumentParser(description='MCU Command Line Interface')
	parser.add_argument('com_port', type=str, help="The COM port to use")
	args = parser.parse_args()

	global ser
	ser = serial.Serial(args.com_port, 115200, timeout = 1)

	print(f"Using COM port: {args.com_port}")

	print("Press CTRL-C to exit the program")	
	try:
		while True:
			cmd = input('enter MCU command: ')
			send_command(cmd)
			get_response()
	except KeyboardInterrupt:
		print("\nWe are done.")
	finally:
		ser.close()

if __name__ == "__main__":
	main()


## Operation

Make the three files. `boot.py` and `code.py` belong on the MCU and `test_serial.py` goes on HOST.  Once the MCU is setup and plugged in, use the device manager to find the correct com port. You will see that the MCU has created TWO com ports and the one with the lower number is likely the one you want. Run the test serial command by typing `python -m test_serial COMXX` on the command line of your computere where COMXX is the COM port you found earlier. 

## Understanding

Open ChatGPT or your favorite AI program and copy one of the scripts into the prompt and ask it to explain the code. Read through the description.





# Expansions and additions

Getting information about com ports using Python instead of the device manager

In [14]:
import serial.tools.list_ports

def list_com_ports():
    ports = serial.tools.list_ports.comports()
    for port in ports:
        print(f"Device: {port.device}")
        print(f"Name: {port.name}")
        print(f"Description: {port.description}")
        print(f"HWID: {port.hwid}")
        print(f"VID: {port.vid}")
        print(f"PID: {port.pid}")
        print(f"Serial Number: {port.serial_number}")
        print(f"Location: {port.location}")
        print(f"Manufacturer: {port.manufacturer}")
        print(f"Product: {port.product}")
        print(f"Interface: {port.interface}")
        print("-" * 40)

if __name__ == "__main__":
    list_com_ports()

Device: COM21
Name: COM21
Description: USB Serial Device (COM21)
HWID: USB VID:PID=0328:0001 SER=B67EF5F7364D47532020204D420A02FF LOCATION=1-3:x.0
VID: 808
PID: 1
Serial Number: B67EF5F7364D47532020204D420A02FF
Location: 1-3:x.0
Manufacturer: Microsoft
Product: None
Interface: None
----------------------------------------
Device: COM20
Name: COM20
Description: USB Serial Device (COM20)
HWID: USB VID:PID=0328:0001 SER=B67EF5F7364D47532020204D420A02FF LOCATION=1-3:x.2
VID: 808
PID: 1
Serial Number: B67EF5F7364D47532020204D420A02FF
Location: 1-3:x.2
Manufacturer: Microsoft
Product: None
Interface: None
----------------------------------------
Device: COM3
Name: COM3
Description: Intel(R) Active Management Technology - SOL (COM3)
HWID: PCI\VEN_8086&DEV_51E3&SUBSYS_0B041028&REV_01\3&11583659&1&B3
VID: None
PID: None
Serial Number: None
Location: None
Manufacturer: Intel
Product: None
Interface: None
----------------------------------------


In [2]:
p = serial.tools.list_ports.comports()

In [4]:
dir(p[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply_usb_info',
 'description',
 'device',
 'hwid',
 'interface',
 'location',
 'manufacturer',
 'name',
 'pid',
 'product',
 'serial_number',
 'usb_description',
 'usb_info',
 'vid']

In [13]:
p[0].apply_usb_info

<bound method ListPortInfo.apply_usb_info of <serial.tools.list_ports_common.ListPortInfo object at 0x000002561C590C10>>